In [3]:
from dask.distributed import Client, progress, LocalCluster
from rechunker import rechunk
import s3fs
import zarr

In [4]:
s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-west-2'))
s3_store = s3fs.S3Map(root='mur-sst/zarr', s3=s3, check=False)
ds_zarr = zarr.open_consolidated(s3_store, mode='r') #, mask_and_scale=False) - Do we wa    nt mask_and_scale here?

In [9]:
ds_subset = ds_zarr['analysed_sst']

# Toy example
subset = ds_zarr['analysed_sst'] \
  .get_orthogonal_selection((slice(0,100), slice(0,100), slice(0,100)))
store = zarr.DirectoryStore('data/example.zarr')
root = zarr.group(store, overwrite=True)
root.create_dataset('analysed_sst', data=subset, chunks=(2,2,2))
source_array = root['analysed_sst']
target_chunks = (10,10,10)
max_mem = '1MB'

In [12]:
target_store = 'rechunked.zarr'
temp_store = 'rechunked-tmp.zarr'
    
array_plan = rechunk(
    source_array,
    target_chunks,
    max_mem,
    target_store,
    temp_store)
array_plan

<Rechunked>
* Source      : <zarr.core.Array '/analysed_sst' (100, 100, 100) int16>
* Target      : <zarr.core.Array (100, 100, 100) int16>

In [14]:
cluster = LocalCluster()
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:42147 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 3 Memory: 6.24 GB


In [17]:
future = array_plan.persist()
progress(future)

VBox()

In [20]:
a = zarr.open('rechunked.zarr')

In [21]:
a.chunks

(10, 10, 10)

In [22]:
import dask.array as dsa
dsa.from_zarr('rechunked.zarr')

,Array,Chunk
Bytes,2.00 MB,2.00 kB
Shape,"(100, 100, 100)","(10, 10, 10)"
Count,1001 Tasks,1000 Chunks
Type,int16,numpy.ndarray


In [27]:
source_group = ds_zarr['analysed_sst']

In [35]:
target_chunks = {
    'analysed_sst': {'time': 379, 'lat': 439, 'lon': 360},
    'time': None, # don't rechunk this array
    'lon': None,
    'lat': None,
}
max_mem = '1GB'

target_store = 'group_rechunked2.zarr'
temp_store = 'group_rechunked-tmp.zarr'

array_plan = rechunk(ds_zarr, target_chunks, max_mem, target_store, temp_store)
array_plan

<Rechunked>
* Source      : <zarr.hierarchy.Group '/' read-only>
* Intermediate: <zarr.hierarchy.Group '/'>
* Target      : <zarr.hierarchy.Group '/'>

In [ ]:
with ProgressBar():
    array_plan.execute()